<a href="https://colab.research.google.com/github/josephx-19/4GeeksAcademy/blob/master/OPTIMIZATION/FeatureSelectionGenetic(BIT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Iván Andrés Trujillo Abella
# ivantrujillo1229@gmail.com

In [37]:
# First we want
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/it-ces/Datasets/main/basesuper.csv")
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import ElasticNet
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import  GridSearchCV
from scipy import stats
from sklearn.preprocessing import StandardScaler

In [38]:
X = ['Ingresos de actividades ordinarias','Costo de ventas',
      'Ganancia (pérdida) antes de impuestos',
     'Total de activos', 'Total pasivos',
      'Patrimonio total', 'Ganancia (pérdida) por actividades de operación',
      'Pasivos corrientes totales',  'Ganancia bruta',
       'Ganancia (pérdida)', 'Ganancias acumuladas', 'y' ]
df  = df[X]

In [39]:
df.dropna(inplace=True)

In [40]:
# how many models you can test?
2**(df.shape[1]-1)

2048

In [41]:
def Xy(df_,target):
    """
    Split the data in X,y to ML implementations
    """
    df = df_.copy()
    X = df.loc[ : , df.columns != target]
    y = df[target]
    return X,y

In [42]:
X,y = Xy(df, 'y')
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1234)
X, y = rus.fit_resample(X, y)

In [43]:
## Testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle = True, random_state = 666, stratify=y)

In [44]:
print(y_train.value_counts())
print(y_test.value_counts())


y
0    109
1    109
Name: count, dtype: int64
y
0    47
1    47
Name: count, dtype: int64


In [45]:
## Write the guided function
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
def score2(solution, X_train_, y_train_):
    X_train, y_train = X_train_.copy(), y_train_.copy()
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.30, shuffle = True,
                                                    random_state = 666, stratify=y_train)
    solution = np.array(solution, dtype='bool')
    X_train = X_train.loc[:, solution].to_numpy()
    X_val = X_val.loc[:, solution].to_numpy()
    return f1_score(y_val, LogisticReg_gridcv(X_train, y_train).predict(X_val)),

def score(solution, X_train_, y_train_):
    X_train, y_train = X_train_.copy(), y_train_.copy()
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.30, shuffle = True,
                                                    random_state = 666, stratify=y_train)
    solution = np.array(solution, dtype='bool')
    X_train = X_train.loc[:, solution].to_numpy()
    X_val = X_val.loc[:, solution].to_numpy()
    return f1_score(y_val, LogisticRegression().fit(X_train, y_train).predict(X_val)),


def grid_lr(X_train, y_train):
    model = LogisticRegression(random_state=42, max_iter=500)

    param_grid = [
        # Solvers que solo admiten 'l2' o None
        {
            'solver': ['newton-cg', 'lbfgs', 'sag'],
            'penalty': ['l2', None],
            'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'class_weight': [None, 'balanced', {0: 0.2, 1: 0.8}, {0: 0.3, 1: 0.7}, {0: 0.4, 1: 0.6}],
        },
        # Solvers que admiten 'l1' y 'l2', sin 'dual' cuando 'l1'
        {
            'solver': ['liblinear'],
            'penalty': ['l2'],
            'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'class_weight': [None, 'balanced', {0: 0.2, 1: 0.8}, {0: 0.3, 1: 0.7}, {0: 0.4, 1: 0.6}],
            'dual': [False, True],  # 'dual' solo se usa con 'l2'
        },
        {
            'solver': ['liblinear'],
            'penalty': ['l1'],
            'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'class_weight': [None, 'balanced', {0: 0.2, 1: 0.8}, {0: 0.3, 1: 0.7}, {0: 0.4, 1: 0.6}],
            'dual': [False],  # 'dual' debe ser False cuando 'l1'
        },
        # Solvers que admiten 'elasticnet'
        {
            'solver': ['saga'],
            'penalty': ['l1', 'l2', 'elasticnet', None],
            'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'class_weight': [None, 'balanced', {0: 0.2, 1: 0.8}, {0: 0.3, 1: 0.7}, {0: 0.4, 1: 0.6}],
            'l1_ratio': [0, 0.1, 0.5, 0.9, 1],  # Solo aplica para 'elasticnet'
        }
    ]

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=42)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        n_jobs=-1,
        cv=cv,
        scoring='roc_auc',
        error_score='raise'
    )

    grid_result = grid_search.fit(X_train, y_train)

    return grid_result.best_estimator_

def LogisticReg_gridcv(X_train, y_train):
    model =  LogisticRegression(solver='saga', random_state=42)
    hyperparams = {"C" :  [0.001, 0.01, 0.1, 1],
                   "penalty" :  ['l1', 'l2', None,"elasticnet"],
                   "class_weight": [None, "balanced", {0: 0.2, 1: 0.8}, {0: 0.3, 1: 0.7}, {0: 0.4, 1: 0.6}],
                   'l1_ratio': [0, 0.1, 0.5, 0.9, 1],
                   "max_iter":[ 100, 120, 150, 200]}
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=123) # replicables...
    grid_search = GridSearchCV(estimator=model,
                               param_grid=hyperparams,
                               cv=cv,
                               scoring= 'roc_auc',)
    grid_result = grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_


In [46]:
sol = [1,1,0,0,1,0,1,1,1,1,0]
score(sol, X_train, y_train)

(0.746268656716418,)

In [47]:
!pip install deap

In [48]:
import random
from deap import creator
from deap import base
from deap import tools
from deap import algorithms
import matplotlib.pyplot as plt

In [49]:
chromosomal_size = X_train.shape[1]

In [50]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("binaryGen", random.randint, 0,1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binaryGen, chromosomal_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, 80)
toolbox.register("evaluate", lambda x: score(x,X_train, y_train))
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3) #change to rolloute wheel
pop = toolbox.population()
pop, logbok = algorithms.eaSimple(pop, toolbox, cxpb=0.35, mutpb=0.05, ngen=60, verbose=False)
print(pop[0])
# Could be very useful graph while is running to see the behavior of the algorithm!!!
# uses method fit to any model of scikit-learn

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]


In [51]:
pop[0]

[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]

In [52]:
toolbox.evaluate(pop[0])

(0.7692307692307693,)

In [53]:
## Add graph

In [54]:
X_train.loc[:,np.array(pop[0], dtype='bool')]

,Ganancia (pérdida) antes de impuestos,Total de activos,Ganancia (pérdida)
11949,1426770.0,62330759.0,1255619.0
10357,3037631.0,12442440.0,1950280.0
5093,596682.0,6372988.0,399777.0
5108,-2266002.0,5413417.0,-2266002.0
957,-608723.0,13632151.0,-676806.0
...,...,...,...
7520,731452.0,21412644.0,596802.0
10312,-24748.0,14745906.0,-82815.0
11451,1132983.0,17651867.0,970638.0
10598,2287994.0,24120741.0,1166834.0


In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("binaryGen", random.randint, 0,1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binaryGen, chromosomal_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, 80)
toolbox.register("evaluate", lambda x: score2(x,X_train, y_train))
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3) #change to rolloute wheel
pop = toolbox.population()
pop, logbok = algorithms.eaSimple(pop, toolbox, cxpb=0.35, mutpb=0.05, ngen=60, verbose=False)
print(pop[0])
# Could be very useful graph while is running to see the behavior of the algorithm!!!
# uses method fit to any model of scikit-learn

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skle

In [ ]:
pop[0]


In [ ]:
toolbox.evaluate(pop[0])

In [ ]:
X_train.loc[:,np.array(pop[0], dtype='bool')]

# Generalize to another models
Uses the starndard mehtods of _scikit-learn_

In [ ]:
########################################
# Genetic Feature Selection function  ##
#########################################

def score(solution, classifier, X_train_, y_train_):
    """
    Classifier must have the fit and predict methods (sklearn)
    """
    X_train, y_train = X_train_.copy(), y_train_.copy()
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.30, shuffle = True,
                                                    random_state = 666, stratify=y_train)
    solution = np.array(solution, dtype='bool')
    X_train = X_train.loc[:, solution].to_numpy()
    X_val = X_val.loc[:, solution].to_numpy()
    return f1_score(y_val, classifier.fit(X_train, y_train).predict(X_val)),

In [ ]:
sol = [0,0,0,1,1,1,0,0,0,1,1]
classifierLR = LogisticRegression(random_state=123) # Here is explicit to change hyperparamters

In [ ]:
score(sol, classifierLR, X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifierDT = DecisionTreeClassifier(random_state=123)

In [ ]:
score(sol, classifierDT, X_train, y_train)

In [ ]:
from sklearn.neural_network import MLPClassifier
classifierMLP = MLPClassifier(random_state=213)

In [ ]:
score(sol, classifierMLP, X_train, y_train)

In [ ]:
def feature_ga(classifier):
    random.seed(123)
    chromosomal_size = X_train.shape[1]
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    toolbox.register("binaryGen", random.randint, 0,1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binaryGen, chromosomal_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual, 80)
    toolbox.register("evaluate", lambda x: score(x,classifier,X_train, y_train))
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3) #change to rolloute wheel
    pop = toolbox.population()
    pop, logbok = algorithms.eaSimple(pop, toolbox, cxpb=0.35, mutpb=0.05, ngen=60, verbose=False)
    return pop[0], toolbox.evaluate(pop[0])
# Could be very useful graph while is running to see the behavior of the algorithm!!!
# uses method fit to any model of scikit-learn

In [ ]:
feature_ga(classifierLR)

In [ ]:
feature_ga(classifierDT)

In [ ]:
feature_ga(classifierMLP)